In [1]:
import pandas as pd
import os

import warnings
warnings.simplefilter("ignore")

In [2]:
path_23 = os.path.join(os.getcwd(), "data", "raw", "01_23-01_24")
path_24 = os.path.join(os.getcwd(), "data", "raw", "01_24-03_24")

path_concorrentes_23 = path_23 + """/soujunior_extração_concorrentes_01_23-01_24.csv"""
path_conteudo_23 = path_23 + """/soujunior_extração_conteúdo_01_23-01_24.xlsx"""
path_seguidores_23 = path_23 + """/soujunior_extração_seguidores_01_23-01_24.xls"""
path_visitantes_23 = path_23 + """/soujunior_extração_visitantes.xlsx"""

path_concorrentes_24 = path_24 + "/concorrentes_01_24-03_24.xlsx"
path_conteudo_24 = path_24 + "/conteudo_01_24-03_24.xls"
path_seguidores_24 = path_24 + "/seguidores_01_24-03_24.xls"
path_visitantes_24 = path_24 + "/visitantes_01_24-03_24.xls"

path_export = os.path.join(os.getcwd(), "data", "processed")


In [3]:
# funções
# lendo as ultimas datas
ultima_data_23 = str(pd.read_excel(path_seguidores_23, sheet_name=0)["Data"].iloc[-1])
ultima_data_24 = str(pd.read_excel(path_seguidores_24, sheet_name=0)["Data"].iloc[-1])

print(f"ultima_data_23: {ultima_data_23}")
print(f"ultima_data_24: {ultima_data_24}")

def read_and_concat_dfs(path1, path2, sheet, drop, skiprows_ = 0):
    df1 = pd.read_excel(path1, sheet_name=sheet, skiprows= skiprows_)
    df2 = pd.read_excel(path2, sheet_name=sheet, skiprows= skiprows_)

    return pd.concat([df1, df2]).drop_duplicates(subset=[drop])


def read_and_merge_dfs(path1, path2, sheet, date1=ultima_data_23, date2=ultima_data_24):
    df1 = pd.read_excel(path1, sheet_name=sheet)
    df2 = pd.read_excel(path2, sheet_name=sheet)

    df1.insert(0, "Data", date1)
    df2.insert(0, "Data", date2)

    df_merged = pd.merge(left=df1, right=df2, how="outer")
    df_merged = df_merged.sort_values(by=df_merged.columns[-1], ascending=False)
    return df_merged

ultima_data_23: 01/16/2024
ultima_data_24: 03/24/2024


In [4]:
# conteudo
df_conteudo_metricas_concat = read_and_concat_dfs(
    path_conteudo_23, path_conteudo_24, 0, "Data", skiprows_=1
)
df_conteudo_publicacoes_concat = read_and_concat_dfs(
    path_conteudo_24, path_conteudo_23, 1, "Link da publicação", skiprows_=1
)

df_conteudo_metricas_concat.to_csv(path_export + "/conteudo_metricas.csv", index=False)
df_conteudo_publicacoes_concat.to_csv(path_export + "/conteudo_publicacoes.csv", index=False)

In [5]:
# seguidores
df_seguidores_novos_concat = read_and_concat_dfs(path_seguidores_23, path_seguidores_24, 0, "Data")

df_seguidores_localidade_merged = read_and_merge_dfs(
    path_seguidores_23, path_seguidores_24, 1
)
df_seguidores_funcao_merged = read_and_merge_dfs(
    path_seguidores_23, path_seguidores_24, 2
)
df_seguidores_experiencia_merged = read_and_merge_dfs(
    path_seguidores_23, path_seguidores_24, 3
)
df_seguidores_setor_merged = read_and_merge_dfs(
    path_seguidores_23, path_seguidores_24, 4
)
df_seguidores_tamanho_empresa_merged = read_and_merge_dfs(
    path_seguidores_23, path_seguidores_24, 5
)

df_seguidores_novos_concat.to_csv(path_export + "/seguidores_novos.csv", index=False)
df_seguidores_localidade_merged.to_csv(path_export + "/seguidores_localidade.csv", index=False)
df_seguidores_funcao_merged.to_csv(path_export + "/seguidores_funcao.csv", index=False)
df_seguidores_experiencia_merged.to_csv(path_export + "/seguidores_experiencia.csv", index=False)
df_seguidores_setor_merged.to_csv(path_export + "/seguidores_setor.csv", index=False)
df_seguidores_tamanho_empresa_merged.to_csv(path_export + "/seguidores_tamanho_empresa.csv", index=False)

In [6]:
# visitantes
df_visitantes_novos_concat = read_and_concat_dfs(path_visitantes_23, path_visitantes_24, 0, "Data")

df_visitantes_localidade_merged = read_and_merge_dfs(
    path_visitantes_23, path_visitantes_24, 1
)
df_visitantes_funcao_merged = read_and_merge_dfs(
    path_visitantes_23, path_visitantes_24, 2
)
df_visitantes_experiencia_merged = read_and_merge_dfs(
    path_visitantes_23, path_visitantes_24, 3
)
df_visitantes_setor_merged = read_and_merge_dfs(
    path_visitantes_23, path_visitantes_24, 4
)
df_visitantes_tamanho_empresa_merged = read_and_merge_dfs(
    path_visitantes_23, path_visitantes_24, 5
)

df_visitantes_novos_concat.to_csv(path_export + "/visitantes_novos.csv", index=False)
df_visitantes_localidade_merged.to_csv(path_export + "/visitantes_localidade.csv", index=False)
df_visitantes_funcao_merged.to_csv(path_export + "/visitantes_funcao.csv", index=False)
df_visitantes_experiencia_merged.to_csv(path_export + "/visitantes_experiencia.csv", index=False)
df_visitantes_setor_merged.to_csv(path_export + "/visitantes_setor.csv", index=False)
df_visitantes_tamanho_empresa_merged.to_csv(path_export + "/visitantes_tamanho_empresa.csv", index=False)

In [7]:
# concorrentes
concorrentes_23 = pd.read_csv(path_concorrentes_23, skiprows=1)
concorrentes_24 = pd.read_excel(path_concorrentes_24, skiprows=1)
concorrentes_23 = concorrentes_23[list(concorrentes_24.columns)]

concorrentes_23.insert(0, "Data", ultima_data_23)
concorrentes_24.insert(0, "Data", ultima_data_24)

concorrentes_concat = pd.concat([concorrentes_23, concorrentes_24])
concorrentes_concat.to_csv(path_export + "/concorrentes.csv", index=False)